In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional,GlobalAveragePooling1D, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import cv2
import numpy as np
import os
import glob
import json
from tensorflow import keras
from sklearn.model_selection import train_test_split

C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\api_core\_python_version_support.py:246: FutureWarning: You are using a non-supported Python version (3.9.10). Google will not post any further updates to google.api_core supporting this Python version. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\__init__.py:54: FutureWarning: 
    You are using a Python version 3.9 past its end of life. Google will update
    google-auth with critical bug fixes on a best-effort basis, but not
    with any other fixes or features. Please upgrade your Python version,
    and then update google-auth.
    
  warnings.warn(eol_message.format("3.9"), FutureWarning)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\google\oauth2\__init__.py:40: FutureWarni

In [2]:
# 1. Các biến cấu hình
DATA_PATH        = 'Data'                # thư mục gốc chứa các folder action
LABEL_MAP_PATH   = 'Logs/label_map.json'
BATCH_SIZE       = 32
AUTOTUNE         = tf.data.AUTOTUNE
VAL_SPLIT        = 0.1
TEST_SPLIT       = 0.1
# 2. Load label_map từ JSON
with open(LABEL_MAP_PATH, 'r', encoding='utf-8') as f:
    label_map = json.load(f)            # ví dụ: {"địa chỉ": 0, "miến điện": 1, ...}

# 3. Tạo danh sách tất cả các file .npz
#    Data structure: Data/<action_name>/*.npz
file_pattern = os.path.join(DATA_PATH, '**', '*.npz')
all_files = glob.glob(file_pattern, recursive=True)
print(f"Found {len(all_files)} samples.")

train_files, temp_files = train_test_split(
    all_files,
    test_size=VAL_SPLIT + TEST_SPLIT,  # ví dụ: 0.2 + 0.1 = 0.3
    shuffle=True,
    random_state=42,
    stratify=[os.path.basename(p).split('.')[0] for p in all_files]
)

val_files, test_files = train_test_split(
    temp_files,
    test_size=TEST_SPLIT / (VAL_SPLIT + TEST_SPLIT),  # ví dụ: 0.1 / 0.3 = 1/3
    shuffle=True,
    random_state=42,
    stratify=[os.path.basename(p).split('.')[0] for p in temp_files]
)

print(f"  Train samples: {len(train_files)}")
print(f"    Val samples: {len(val_files)}")
print(f"   Test samples: {len(test_files)}")

# 4. Hàm parse mỗi file .npz
def _load_npz(path):
    # path: scalar tf.string tensor
    npz_path = path.decode('utf-8')
    data = np.load(npz_path)
    seq   = data['sequence'].astype(np.float32)   # (60,126)
    lbl   = np.int32(data['label'])
    return seq, lbl

def parse_fn(path):
    seq, lbl = tf.numpy_function(
        func=_load_npz,
        inp=[path],
        Tout=[tf.float32, tf.int32]
    )
    # set shape để TF biết kích thước cố định
    seq.set_shape([60, 201])
    lbl.set_shape([])
    return seq, lbl
def make_dataset(file_list, shuffle=False, repeat=False):
    ds = tf.data.Dataset.from_tensor_slices(file_list)
    if shuffle:
        ds = ds.shuffle(len(file_list), reshuffle_each_iteration=True)
    if repeat:
        ds = ds.repeat()
    ds = ds.map(parse_fn, num_parallel_calls=AUTOTUNE)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(AUTOTUNE)
    return ds

# 6. Tạo train_ds & val_ds
train_ds = make_dataset(train_files, shuffle=True, repeat=True)
val_ds   = make_dataset(val_files, shuffle=False, repeat=False)
test_ds  = make_dataset(test_files, shuffle=False, repeat=False)

# 7. Compute steps
steps_per_epoch = len(train_files) // BATCH_SIZE
validation_steps = len(val_files) // BATCH_SIZE

Found 36006 samples.
  Train samples: 28804
    Val samples: 3601
   Test samples: 3601


In [3]:
inputs = tf.keras.Input(shape=(60, 201))

# Khối LSTM thứ nhất
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3))(inputs)
x = BatchNormalization()(x)

# Khối LSTM thứ hai
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3))(x)
x = BatchNormalization()(x)

# Khối LSTM thứ ba
x = Bidirectional(LSTM(256, dropout=0.3))(x)
x = BatchNormalization()(x)

# Các lớp Dense
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)

# Lớp đầu ra
outputs = Dense(2764, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
# Biên dịch mô hình
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [4]:
# 1. Tạo thư mục lưu checkpoint (nếu chưa có)
checkpoint_dir = 'Models/checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'final_model.keras')

# 2. Khởi tạo callbacks
callbacks = [
    # Lưu mô hình với val_loss thấp nhất
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,  # lưu cả kiến trúc + weights
        verbose=1
    ),
    # Dừng training nếu 5 epoch liên tiếp không cải thiện val_loss
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
]

In [ ]:
model.fit(
    train_ds,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks = callbacks
)

Epoch 1/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0041 - loss: 6.7241
Epoch 1: val_loss improved from inf to 5.08210, saving model to Models/checkpoints\final_model.keras
900/900 ━━━━━━━━━━━━━━━━━━━━ 1911s 2s/step - accuracy: 0.0041 - loss: 6.7234 - val_accuracy: 0.0254 - val_loss: 5.0821
Epoch 2/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 1903s 2s/step - accuracy: 0.0271 - loss: 5.1634
Epoch 3/100


C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
C:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0855 - loss: 4.2056
Epoch 3: val_loss improved from 5.08210 to 2.97766, saving model to Models/checkpoints\final_model.keras
900/900 ━━━━━━━━━━━━━━━━━━━━ 2003s 2s/step - accuracy: 0.0855 - loss: 4.2053 - val_accuracy: 0.2511 - val_loss: 2.9777
Epoch 4/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 1836s 2s/step - accuracy: 0.1834 - loss: 3.2675
Epoch 5/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2920 - loss: 2.5598
Epoch 5: val_loss improved from 2.97766 to 1.31618, saving model to Models/checkpoints\final_model.keras
900/900 ━━━━━━━━━━━━━━━━━━━━ 2003s 2s/step - accuracy: 0.2920 - loss: 2.5596 - val_accuracy: 0.6069 - val_loss: 1.3162
Epoch 6/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 2137s 2s/step - accuracy: 0.4100 - loss: 1.9795
Epoch 7/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4880 - loss: 1.6415
Epoch 7: val_loss improved from 1.31618 to 0.78757, saving model to Models/checkpoints\final_model.keras
900/900 ━━━━━━━━━━━━━━━

900/900 ━━━━━━━━━━━━━━━━━━━━ 1962s 2s/step - accuracy: 0.9324 - loss: 0.2077
Epoch 37/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9294 - loss: 0.2226
Epoch 37: val_loss did not improve from 0.04108
900/900 ━━━━━━━━━━━━━━━━━━━━ 2175s 2s/step - accuracy: 0.9294 - loss: 0.2226 - val_accuracy: 0.9729 - val_loss: 0.0817
Epoch 38/100
900/900 ━━━━━━━━━━━━━━━━━━━━ 2255s 3s/step - accuracy: 0.8754 - loss: 0.3881
Epoch 43/100
 89/900 ━━━━━━━━━━━━━━━━━━━━ 37:56 3s/step - accuracy: 0.8998 - loss: 0.3206